### 비정형 데이터 융합형 기업 예측 모델
* 뉴스나 전문가 의견및 여론 기반한 비정형 데이터 수집
* 주가 등 정형 데이터 수집
* 기업의 주가 영향 요소 예측등을 통한
* 기업의 위험관리및 투자예측 분석

설립일:1955년 12월 9일 (65주년)[2]
주소
서울특별시 서초구 신반포로 194 (반포동)[3]
업종명
백화점
상장유무
상장기업
기업규모
대기업
상장시장
유가증권시장 (1985년 ~ 현재)
대표이사
차정호
종목코드
004170
자본금
492억 2천만원 (2019)
매출액
1조 5,576억원 (2019)
영업이익
2,220억 9천만원 (2019)
당기순이익
7,059억 5천만원 (2019)
직원 수
2,745명 (2020.06.30)

### 분석요소
* 뉴스 :
* SNS  :
* 댓글 :
* 재무재표: 
* 기업분석 지표:

* 제품 매출 현황

* 시가 :
* 종가 :
* 고가 :
* 저가 :
* 거래량:

### 네이버 뉴스 수집
* 신세계
* url = https://search.naver.com/search.naver?query=%EC%8B%A0%EC%84%B8%EA%B3%84&where=news&ie=utf8&sm=nws_hty

In [1]:
url='https://search.naver.com/search.naver?&where=news&query={0}&start={1}&pd=3&ds=2020.01.01&de=2020.12.31'.format('신세계',11)
url

'https://search.naver.com/search.naver?&where=news&query=신세계&start=11&pd=3&ds=2020.01.01&de=2020.12.31'

In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bsp
import matplotlib
import matplotlib.pyplot as plt
import sqlite3
import re
from datetime import datetime, timedelta,date
import sys

In [3]:
def calcDate(txt=''):
    #print(txt)
    rdate=''
    nowd=datetime.now()
    pattern='[^\d]'
    num=int(re.sub(pattern,'',txt))
    if('일' in txt):
        res=nowd + timedelta(days=num*-1)
    elif('.' in txt):
        ds=txt.replace('.','-')
        #print(ds)
        res=ds+' 00:00:00.000000'
    else:
        res=nowd + timedelta(hours=num*-1)
    return res

In [4]:
calcDate('2021.2.1')

'2021-2-1 00:00:00.000000'

In [5]:
def makeKey(txt=''):
    txt_=re.sub('http\S*//',' ',txt)
    key=re.sub('[^a-zA-Z0-9%]','',txt_)
    return key

In [6]:
def getData(whr=''):
    dbname='navernews.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select * from nnews '+whr
        res=cursor.execute(sql).fetchall()
    return(res)

In [7]:
def insMain(d):
    dbname='navernews.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select * from nnews where nkey="'+d[5]+'"'
        res=cursor.execute(sql).fetchall()
        if(len(res)>0):
            print("이미 존재 합니다.:"+d[1])
        else:
            sql='insert into nnews (ncate,ntitle,ncomp,ndate,nurl,nkey) values(?,?,?,?,?,?)'
            res=cursor.execute(sql,d)
            print('.',end='')
        

In [8]:
def getPage(url):
    res=requests.get(url)
    src=res.content.strip()
    htmls=bsp(src,'html.parser')
    print(url)
    tgts=htmls.select('ul.list_news > li')
    if(len(tgts)==0):
        print('데이터가 없습니다.'+url)
        sys.exit(1)
    for tgt in tgts:
        mainarti=tgt.select('div.news_wrap')
        subartis=tgt.select('ul.list_cluster > li.sub_bx')
        if(len(mainarti)>0):
            title_= mainarti[0].select('a.news_tit')[0].text
            title=re.sub('[^ㄱ-힣0-9 a-zA-Z]',' ',title_)
            comp=mainarti[0].select('.info.press')[0].text.replace('언론사 선정','')
            date_=mainarti[0].select('span.info')[0].text.replace('시간 전','')
            date=calcDate(date_)
            url=mainarti[0].select('a.news_tit')[0]['href']
            key=makeKey(url)
            opt=1
            d=(opt,title,comp,date,url,key)
            insMain(d)
            for subarti in subartis:
                title_= subarti.select('a.sub_tit')[0].text
                title=re.sub('[^ㄱ-힣0-9 a-zA-Z]',' ',title_)
                comp=subarti.select('.sub_txt.press')[0].text.replace('언론사 선정','')
                date_=subarti.select('span.sub_txt')[0].text.replace('시간 전','').replace('네이버뉴스','')
                date=calcDate(date_)
                url=subarti.select('a.sub_tit')[0]['href']
                key=makeKey(url)
                opt=2
                d=(opt,title,comp,date,url,key)
                insMain(d)



In [9]:
urlsrc1='https://search.naver.com/search.naver?&where=news&query={0}&start={1}'
urlsrc='https://search.naver.com/search.naver?&where=news&query={0}&start={1}&pd=3&ds=2019.01.01&de=2019.12.31'
for i in range(100):
    url=urlsrc.format('신세계',i*10+1)
    print(url)
    getPage(url)

https://search.naver.com/search.naver?&where=news&query=신세계&start=1&pd=3&ds=2019.01.01&de=2019.12.31
https://search.naver.com/search.naver?&where=news&query=신세계&start=1&pd=3&ds=2019.01.01&de=2019.12.31
이미 존재 합니다.:신세계면세점  中 관광객 겨냥 위챗 미니앱 내 여행서비스 앱 탑재
이미 존재 합니다.:신세계면세점  위챗 미니앱 내 여행 콘텐츠 탑재 中 싼커 공략
이미 존재 합니다.:신세계免  위챗 미니앱 서비스 내 여행 콘텐츠 탑재
이미 존재 합니다.:신세계免  위챗 미니앱에  디스커버서울  탑재
이미 존재 합니다.:신세계면세점  위챗 미니앱 서비스 내 여행컨텐츠 탑재
이미 존재 합니다.:신세계免  음악 인재 후원금 3000만원 전달
이미 존재 합니다.:신세계면세점  서초구 중고등학생  클래식 음악 영재 장학금  기탁
이미 존재 합니다.:신세계면세점  서초구 클래식  음악 영재  장학금 기탁
이미 존재 합니다.:신세계면세점 서초구중고등학생을 위한 클래식음악 영재장학금기탁
이미 존재 합니다.:신세계면세점  서초구 중고등학생 위한 클래식 음악 영재 장학금 기탁
이미 존재 합니다.:신세계푸드  흰 쥐 모티브 케이크 출시
이미 존재 합니다.:신세계푸드  경자년 기념 케이크 5종 출시
이미 존재 합니다.:신세계푸드   흰 쥐의 해  맞아 베이커리 신제품 5종 출시
이미 존재 합니다.:신세계푸드   흰 쥐의 해  맞아 베이커리 신제품 출시
이미 존재 합니다.:신세계푸드  흰쥐의 해 맞아 베이커리 신제품 출시
이미 존재 합니다.:신세계百 면세점  경자년 맞이  흰 쥐 모티브 상품  선봬
이미 존재 합니다.:신세계百 면세점  쥐의 해 맞아 쥐 캐릭터 상품 선보여
이미 존재 합니다.:신세계  2020년 쥐띠 해 맞아 쥐 모티브 상품 선봬
이미 존재 합니다.:신세계百  경자년 맞아 쥐 캐릭터 케이크 골드바 마련
이미

KeyboardInterrupt: 